## Lab 4 - Cross-encoder re-ranking

In [1]:
from helper_utils import load_chroma, word_wrap, project_embeddings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
import numpy as np

In [2]:
embedding_function = SentenceTransformerEmbeddingFunction()

chroma_collection = load_chroma(filename='microsoft_annual_report_2022.pdf', collection_name='microsoft_annual_report_2022', embedding_function=embedding_function)
chroma_collection.count()

349

# Re-ranking the long tail

In [3]:
query = "What has been the investment in research and development?"
results = chroma_collection.query(query_texts=query, n_results=10, include=['documents', 'embeddings'])

retrieved_documents = results['documents'][0]

print(f"Query:\n{query}\n")

print(f"Results:")
for document in results['documents'][0]:
    print(word_wrap(document))
    print('')

Query:
What has been the investment in research and development?

Results:
• operating expenses increased $ 1. 5 billion or 14 % driven by
investments in gaming, search and news advertising, and windows
marketing. operating expenses research and development ( in millions,
except percentages ) 2022 2021 percentage change research and
development $ 24, 512 $ 20, 716 18 % as a percent of revenue 12 % 12 %
0ppt research and development expenses include payroll, employee
benefits, stock - based compensation expense, and other headcount -
related expenses associated with product development. research and
development expenses also include third - party development and
programming costs, localization costs incurred to translate software
for international markets, and the amortization of purchased software
code and services content. research and development expenses increased
$ 3. 8 billion or 18 % driven by investments in cloud engineering,
gaming, and linkedin. sales and marketing

competitiv

In [4]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [5]:
pairs = [[query, doc] for doc in retrieved_documents]
scores = cross_encoder.predict(pairs)
print("Scores:")
for score in scores:
    print(score)

Scores:
0.98693424
2.6445792
-0.26803133
-10.731592
-7.7066073
-5.6469946
-4.2970333
-10.933233
-7.038428
-7.324693


In [6]:
print("New Ordering (low to high):")
for o in np.argsort(scores)[::-1]:
    print(o+1)

New Ordering (low to high):
2
1
3
7
6
9
10
5
4
8


# Re-ranking with Query Expansion

In [7]:
original_query = "What were the most important factors that contributed to increases in revenue?"
generated_queries = [
    "What were the major drivers of revenue growth?",
    "Were there any new product launches that contributed to the increase in revenue?",
    "Did any changes in pricing or promotions impact the revenue growth?",
    "What were the key market trends that facilitated the increase in revenue?",
    "Did any acquisitions or partnerships contribute to the revenue growth?"
]

In [8]:
queries = [original_query] + generated_queries

results = chroma_collection.query(query_texts=queries, n_results=10, include=['documents', 'embeddings'])
retrieved_documents = results['documents']

In [9]:
# Deduplicate the retrieved documents
unique_documents = set()
for documents in retrieved_documents:
    for document in documents:
        unique_documents.add(document)

unique_documents = list(unique_documents)

In [10]:
print(f"Query:\n{original_query}\n")

print(f"Results:")

pairs = []
for doc in unique_documents:
    pairs.append([original_query, doc])
    print(doc)
    print("")
    

Query:
What were the most important factors that contributed to increases in revenue?

Results:
productivity and business processes $ 29, 687 $ 24, 351 $ 18, 724 intelligent cloud 32, 721 26, 126 18, 324 more personal computing 20, 975 19, 439 15, 911 total $ 83, 383 $ 69, 916 $ 52, 959 no sales to an individual customer or country other than the united states accounted for more than 10 % of revenue for fiscal years 2022, 2021, or 2020. revenue, classified by the major geographic areas in which our customers were located, was as follows : ( in millions ) year ended june 30, 2022 2021 2020 united states ( a ) $ 100, 218 $ 83, 953 $ 73, 160 other countries 98, 052 84, 135 69, 855 total $ 198, 270 $ 168, 088 $ 143, 015 ( a ) includes billings to oems and certain multinational organizations because of the nature of these businesses and the impracticability of determining the geographic source of the revenue.

( in millions, except percentages ) 2022 2021 percentage change sales and marketi

In [11]:
scores = cross_encoder.predict(pairs)


In [12]:
print("Scores:")
for score in scores:
    print(score)

Scores:
-9.357723
-7.7541003
-10.083942
-7.404206
-1.136996
-9.768026
-3.7681534
-5.27475
-5.1418324
-10.148885
-3.7948625
-10.711212
-8.505106
-9.9184265
-11.079268
-4.8184834
-10.042842
-9.8078785
-7.917178
-6.902088
-4.3417687
-4.6518903


In [13]:
print("New Ordering (Low to high scores):")
for o in np.argsort(scores)[::-1]:
    print(o)

New Ordering (Low to high scores):
4
6
10
20
21
15
8
7
19
3
1
18
12
0
5
17
13
16
2
9
11
14
